# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-cou

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/11/11/ai-live-event/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
htt

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'portfolio project page',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio project page',
   'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'product page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [11]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 6 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin page', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links


{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [13]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 12 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
moonshotai/Kimi-K2-Thinking
Updated
4 days ago
•
89.5k
•
1.07k
maya-research/maya1
Updated
about 14 hours ago
•
15.4k
•
498
baidu/ERNIE-4.5-VL-28B-A3B-Thinking
Updated
about 6 hours ago
•
1.52k
•
305
dx8152/Qwen-Edit-2509-Multiple-angles
Updated
about 6 hours ago
•
34.6k
•
526
MiniMaxAI/MiniMax-M2
Updated
5 days ago
•
886k
•
1.27k
Browse 1M+ models
Spaces
Running
on
CPU Upgrade
2.08k
2.08k
The Smol Training Playbook
📚
The secrets to building world-class LLMs
Running
on
Zero
618
618
Qwen Image Edit Camera Control
🎬
Fast 4 step inferen

In [27]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2-Thinking\nUpdated\n4 days ago\n•\n89.5k\n•\n1.07k\nmaya-research/maya1\nUpdated\nabout 14 hours ago\n•\n15.4k\n•\n498\nbaidu/ERNIE-4.5-VL-28B-A3B-Thinking\nUpdated\nabout 6 hours ago\n•\n1.52k\n•\n305\ndx8152/Qwen-Edit-2509-Multiple-angles\nUpdated\nabout 6 hours ago\n•\n34.6k\n•\n526\nMiniMaxAI/MiniMax-M2\nUpdated\n5 days ago\n•\n886k\n•\n1.27k\nBrowse 1M+ models\nSpaces

In [23]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    print(f"Created brochure with 2nd API call by calling gpt-4.1-mini")
    display(Markdown(result))

In [24]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 8 relevant links
Created brochure with 2nd API call by calling gpt-4.1-mini


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future — a vibrant platform where the machine learning community collaborates on models, datasets, and applications. Serving as the home of machine learning, Hugging Face empowers engineers, scientists, and AI enthusiasts globally to create, discover, and share cutting-edge machine learning projects in an open and ethical environment.

---

## What We Offer

- **Extensive Model Library**: Access over 1 million open-source machine learning models spanning various modalities including text, image, video, audio, and even 3D.
- **Rich Dataset Repository**: Explore and contribute to a collection of 250,000+ datasets to fuel your machine learning innovations.
- **Spaces**: Host and discover 400,000+ AI applications in a collaborative environment, accelerating experimentation and deployment.
- **Open-Source Stack**: Move faster with the Hugging Face open-source tools designed for ease of use and scalability.
- **Enterprise Solutions**: Leverage paid Compute and Enterprise plans providing advanced infrastructure and support for teams and organizations.

---

## Our Community & Culture

Hugging Face fosters an inclusive, collaborative community dedicated to building an open and ethical AI future. Through the Hugging Face Hub, individuals from diverse backgrounds share, explore, and experiment with machine learning models and datasets, growing together as a community. The platform encourages knowledge sharing and portfolio building, making it a dynamic space for learning and career growth.

---

## Who Uses Hugging Face?

Our platform is trusted by a global audience including:

- Machine learning engineers and researchers seeking state-of-the-art open-source models.
- Data scientists looking for diverse datasets and tools to accelerate their projects.
- AI startups and enterprises needing scalable infrastructure and team collaboration features.
- Educators and students aiming to build portfolios and engage with AI applications.
- Developers creating advanced multimodal AI solutions in text, vision, audio, and more.

---

## Careers at Hugging Face

Join a fast-growing, innovative team focused on building the future of AI. Hugging Face values creativity, collaboration, and ethical AI development. We are constantly looking for passionate talent across engineering, research, product, and community roles. By joining Hugging Face, you will contribute to open-source projects that are transforming the machine learning landscape worldwide.

Keep an eye on our [careers page](https://huggingface.co) for job openings and opportunities to be part of a mission-driven company shaping the future of artificial intelligence.

---

## Get Involved

- **Explore Models, Datasets, and Applications**: Visit the Hugging Face Hub to start exploring or sharing your machine learning projects.
- **Join the Community**: Connect with thousands of researchers and developers collaborating in forums, discussions, and open projects.
- **Accelerate Your ML Projects**: Utilize our Compute and Enterprise solutions for scalable, performant AI workflows.

---

## Brand Identity

- **Colors**: Bright, energetic palette anchored by #FFD21E and #FF9D00, symbolizing innovation and approachability.
- **Logo Assets**: Available in scalable vector and raster formats for consistent branding usage.

---

## Contact & More Information

**Website**: [https://huggingface.co](https://huggingface.co)

Join us in building the future of AI — open, ethical, and collaborative.

---

*Hugging Face – The AI community building the future.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 5 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future — a vibrant collaboration platform where the machine learning community comes together to create, share, and innovate. As the home of machine learning, Hugging Face hosts over **1 million models**, **250k+ datasets**, and **400k+ applications** spanning multiple modalities, including text, images, video, audio, and even 3D.

Through its open-source hub, Hugging Face empowers ML engineers, scientists, and enthusiasts to explore and experiment with state-of-the-art models and datasets, accelerating the pace of AI advancement toward an open, ethical future.

---

## What We Offer

- **Models:** Access and contribute to a vast repository of over one million machine learning models across a wide spectrum of applications.
- **Datasets:** Discover and share high-quality datasets, currently numbering above 250,000, supporting everything from autonomous vehicles to language recognition.
- **Spaces:** Host and collaborate on interactive machine learning applications within a shared computational environment.
- **Compute & Enterprise Solutions:** Unlock enhanced performance with paid compute resources and enterprise-grade services tailored for teams and organizations.

---

## Our Community & Culture

At the heart of Hugging Face is a **fast-growing global community** that believes in open collaboration, transparency, and the ethical use of AI. We foster a culture of innovation, inclusivity, and continuous learning where anyone — from novices to experts — can contribute, build their portfolio, and advance their knowledge in machine learning.

Our platform encourages sharing work publicly to enable a collective acceleration of AI capabilities, making ML accessible, beneficial, and trustworthy for all.

---

## Customers & Users

Our diverse user base ranges from independent researchers to global enterprises, leveraging Hugging Face to:

- Develop cutting-edge natural language processing (NLP) and computer vision solutions.
- Train and fine-tune large language models (LLMs).
- Collaborate on multi-modal AI applications combining text, audio, video, and 3D.
- Build production-ready ML pipelines with seamless scalability and enterprise support.

---

## Careers at Hugging Face

Join a mission-driven company at the forefront of AI innovation! We offer exciting opportunities for talented individuals passionate about machine learning, open-source software, and ethical AI development.

**Why Work With Us?**

- Be part of a collaborative, empowering, and inclusive tech community.
- Work on impactful projects that shape the future of AI.
- Opportunity to contribute to open-source technologies used worldwide.
- Competitive benefits and support for professional growth.

Explore career openings and help build the AI community shaping tomorrow’s technology.

---

## Get Involved

- **Explore AI Apps & Models:** Visit [huggingface.co](https://huggingface.co) to dive into thousands of AI applications and models.
- **Join the Community:** Share your ML projects, collaborate with peers, and contribute to open source.
- **Build Your Portfolio:** Showcase your models, datasets, and applications to a global audience.
- **Accelerate Your ML Projects:** Utilize our Compute and Enterprise solutions for faster training and deployment.

---

## Contact & More Information

Website: [https://huggingface.co](https://huggingface.co)  
Join the community, explore models, datasets, spaces, and apply for careers today!

---

*Hugging Face — Building the future of AI, together.*

In [28]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face: Where AI Gets Its Hug On 🤗

---

## Welcome to the Future, Powered by Hugging Face!

Forget lonely coding marathons and secretive AI experiments. At **Hugging Face**, we're all about **community**, **collaboration**, and a sprinkle of good old-fashioned open-source spirit. Think of us as the bustling coffeehouse for machine learning minds where models, datasets, and AI apps meet, mingle, and make magic happen.

---

## What’s the Big Deal? 🤔

- **1 Million+ Models**: From chatty bots to image wizards, they've all set up shop in our hub—ready for you to browse, borrow, or build on.
- **250,000+ Datasets**: Because AI loves a good snack as much as the next genius.
- **400,000+ Applications**: Real projects pushed forward by the community just like you.
- **Spaces**: Launch your experiments and AI demos into the cloud quicker than you can say "neural network."

---

## Why Hugging Face? Because We’re More Than Code 🧡

- **Open & Ethical AI**: We blend cutting-edge technology with the belief that AI should be open, fair, and for everyone.
- **Multimodal Capabilities**: Text, images, video, audio, even 3D — your wildest AI dreams? We support ‘em all.
- **Community-Powered**: Whether you’re a curious coder, a seasoned data scientist, or an AI enthusiast, our platform lets you learn, share, and grow.
- **Build Your Portfolio**: Show off your models, datasets, and apps to the world (and potential employers). Because bragging rights are important.

---

## For Our Awesome Users

- **Machine Learning Engineers & Researchers**: Share your latest groundbreaking model or dataset, find collaborators, and accelerate your work with cutting-edge compute options.
- **Enterprises**: Need enterprise-grade power and support? We've got your back with scalable paid compute and tailored solutions.
- **AI Hobbyists & Educators**: Explore open resources and jump into projects with a welcoming community.

---

## Culture & Careers: Join the AI Revolution

At Hugging Face, we believe in an open workplace where ideas flow freely, creativity blooms, and every ‘what if’ leads to ‘why not?’ We’re a bunch of passionate collaborators, humorists (some say, pun-believers), and AI optimists on a shared mission to build the future.

**Careers?** Yep, we’re hiring! If you want to build world-class AI systems, collaborate with top talent, and make a meaningful impact, check out our roles. Bonus points if you love open source, community vibes, and the occasional cheeky AI pun.

---

## The Essentials (With a Splash of Style)

- **Colors**: Bold and bright — #FFD21E (Hugging Yellow), #FF9D00 (Warm Orange), and grounded #6B7280 (Sophisticated Grey).
- **Logos**: Recognizable, friendly, with that trademark smiley face that says "AI is your friend."

---

## Ready to Hug the Future? 🤗

Dive into the world’s most vibrant AI community. Sign up for free, browse millions of models, contribute your own, and accelerate your machine learning journey with Hugging Face.

**Explore AI apps, join the community, and let's build the future together — one model at a time.**

---

### Hugging Face  
*Where AI Meets Heart & Code*  

*Visit us at* [huggingface.co](https://huggingface.co) and join the revolution today!

---

*P.S. We promise the hugging is virtual—but the impact? Oh, it’s very real.*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>